In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow import keras
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_folder_path = "/content/drive/Shareddrives/ML-final project/data/"
d1 = "abalon.csv"
d2 = "annealing.csv"
d3 = "arrhythmia.csv"
d4 = "audiology-std.csv"
d5 = "autos.csv"
d6 = "balance-scale.csv"
d7 = "baseball.csv"
d8 = "car.csv"
d9 = "cardiotocography-3clases.csv"
d10 = "cardiotocography-10clases.csv"
d11 = "conn-bench-vowel-deterding.csv"
d12 = "contrac.csv"
d13 = "dermatology.csv"
d14 = "ecoli.csv"
d15 = "energy-y1.csv"
d16 = "energy-y2.csv"
d17 = "flags.csv"
d18 = "glass.csv"
d19 = "heart-va.csv"
d20 = "iris.csv"

In [ ]:
#pre-processing
data_folder_path = "/content/drive/Shareddrives/ML-final project/data/"
data_names = [d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20]
data_frames = []
for csv_name in data_names:
  temp_df = pd.read_csv(data_folder_path + csv_name)
  temp_df = temp_df.rename(columns = {'clase': 'class', 'symboling': 'class', 'Hall_of_Fame': 'class'}, inplace = False)
  temp_df = temp_df.fillna(temp_df.mean())
  for col_name in temp_df.columns:
    if temp_df[col_name].dtype == "object":
      temp_df[col_name] = pd.Categorical(temp_df[col_name])
      temp_df[col_name] = temp_df[col_name].cat.codes
  X = temp_df.drop('class', axis=1)
  y = temp_df['class']
  data_frames.append((X,y,len(pd.unique(temp_df['class']))))
  print(csv_name)
  print(temp_df['class'].value_counts())
  print(X.shape)
  print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

abalon.csv
2    1447
0    1407
1    1323
Name: class, dtype: int64
(4177, 8)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
annealing.csv
2    684
1     99
3     67
4     40
0      8
Name: class, dtype: int64
(898, 31)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
arrhythmia.csv
0     245
9      50
1      44
5      25
12     22
3      15
2      15
4      13
8       9
11      5
10      4
6       3
7       2
Name: class, dtype: int64
(452, 262)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
audiology-std.csv
0     47
4     45
1     21
13    20
3     18
10     8
16     8
14     4
7      4
6      4
11     3
2      2
17     2
5      2
9      2
12     2
15     2
8      2
Name: class, dtype: int64
(196, 59)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
autos.csv
 0    67
 1    54
 2    32
 3    27
-1    22
-2     3
Name: class, dtype: int64
(205, 25)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
balance-scale.csv
2    288
1    288
0     49
Name: class, dtype: int64
(625, 4)
xxxxxxxxxx

In [ ]:
#auxillary functions
def get_tpr(big_matrix):
  sum = 0
  for matrix in big_matrix:
    tp = matrix[1][1]
    fn = matrix[1][0]
    sum += (tp/(tp+fn))
  return sum/len(big_matrix)

def get_fpr(big_matrix):
  sum = 0
  for matrix in big_matrix:
    fp = matrix[0][1]
    tn = matrix[0][0]
    sum += (fp/(fp+tn))
  return sum/len(big_matrix)

def convert_to_array(X_train):
  new_x = []
  X_train_1 = np.array(X_train)
  for item in X_train_1:
    new_x.append(item)
  return  np.array(new_x)
  

def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

AUC_PR = keras.metrics.AUC(
    num_thresholds=200,
    curve="PR",
    summation_method="interpolation",
    name="PR",
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False,
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score,auc,recall_score, average_precision_score, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import time

#parameters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

total_csv = "Dataset Name, Algorithm Name, Cross Validation, Hyper Parameters Values, Accuracy, TPR, FPR," \
            " Precision, AUC, PR-Curve, Training Time, Inference Time\n"

num_folds_outter = 10 ## 10 k-folds
kfold_outter = KFold(n_splits=num_folds_outter, shuffle=True)
for index,df in enumerate(data_frames):
  print('=========================================',"data# :",index , "name:" , data_names[index][:-4])
  cv_index = 0
  for train_valid, test in kfold_outter.split(df[0], df[1]):
    print(" cv index: " ,cv_index)
    clf = RandomForestClassifier()
    start_time = time.time()
    clf = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, cv = 3, verbose=0, random_state=42) # 3- folds
    clf = clf.fit(df[0].iloc[train_valid], df[1].iloc[train_valid])
    stop_time = time.time()
    training_time = stop_time - start_time
    print(f"Training time: {stop_time - start_time}s")

    start_time = time.time()
    predictions = clf.predict(df[0].iloc[test])
    stop_time = time.time()
    inference_time = stop_time - start_time
    print(f"inference time: {stop_time - start_time}s")

    y_test = df[1].iloc[test]

    print("hyper parameter values:" , clf.best_params_)

    acc = accuracy_score(y_test, predictions)
    print("accuracy:" ,acc)

    prec = metrics.precision_score(y_test, predictions,average='micro')
    print("precision:" ,prec)
    confusion_matrix = metrics.multilabel_confusion_matrix(y_test, predictions)
    tpr = get_tpr(confusion_matrix)
    fpr = get_fpr(confusion_matrix)
    print("tpr:", tpr )
    print("fpr:", fpr)
    y_test_one_hot = indices_to_one_hot(convert_to_array(y_test.values),df[2])
    y_pred_one_hot = indices_to_one_hot(convert_to_array(predictions),  df[2])
    auc_score = ""
    try:
      auc_score =  metrics.roc_auc_score(y_test_one_hot, y_pred_one_hot,average='macro')
      print("auc:",auc_score)
    except:
      auc_score = "Error"
      print("auc:" "Error")
    pr_curve = AUC_PR(y_test_one_hot,y_pred_one_hot)
    print("PR-Curve", pr_curve)

    csv_line =  data_names[index][:-4] + "," + "Random Forest" + "," + str(cv_index) + "," + str(clf.best_params_) + "," + str(acc) + "," + str(tpr) + "," + str(fpr) + "," + str(prec) + "," + str(auc_score) + "," + str(pr_curve)+ "," +str(training_time) + "," + str(inference_time) + "\n"
    total_csv += csv_line
    cv_index += 1

with open('report.csv', 'wb') as file:
    file.write(total_csv)

from google.colab import files
files.download('report.csv') 

========================================= data# : 0 name: abalon
 cv index:  0
Training time: 137.89938950538635s
inference time: 0.03128385543823242s
hyper parameter values: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}
accuracy: 0.6339712918660287
precision: 0.6339712918660287
tpr: 0.633914306463326
fpr: 0.18331790020569272
auc: 0.7252982031288165
PR-Curve tf.Tensor(0.70399034, shape=(), dtype=float32)
 cv index:  1
Training time: 137.74315977096558s
inference time: 0.035039424896240234s
hyper parameter values: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': True}
accuracy: 0.6483253588516746
precision: 0.6483253588516746
tpr: 0.6422170170840072
fpr: 0.17543334042293648
auc: 0.7333918383305353
PR-Curve tf.Tensor(0.70124435, shape=(), dtype=float32)
 cv index:  2
Training time: 137.67095136642456s
inference time: 0.0303645133972

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  


Training time: 46.37494921684265s
inference time: 0.13056111335754395s
hyper parameter values: {'n_estimators': 1400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}
accuracy: 0.9666666666666667
precision: 0.9666666666666667
tpr: 0.735
fpr: 0.020620842572062086
auc: 0.857189578713969
PR-Curve tf.Tensor(0.6863715, shape=(), dtype=float32)
 cv index:  4
Training time: 45.669283866882324s
inference time: 0.09258317947387695s
hyper parameter values: {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 80, 'bootstrap': False}
accuracy: 0.9888888888888889
precision: 0.9888888888888889
tpr: 0.975
fpr: 0.010526315789473684
auc: 0.9822368421052632
PR-Curve tf.Tensor(0.68716913, shape=(), dtype=float32)
 cv index:  5
Training time: 44.625250816345215s
inference time: 0.037835121154785156s
hyper parameter values: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 1, 'm

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


Training time: 80.45129776000977s
inference time: 0.0919497013092041s
hyper parameter values: {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 80, 'bootstrap': False}
accuracy: 0.6304347826086957
precision: 0.6304347826086957
tpr: nan
fpr: 0.045761637294818074
auc:Error
PR-Curve tf.Tensor(0.6900335, shape=(), dtype=float32)
 cv index:  1


...

Training time: 40.348679065704346s
inference time: 0.16312074661254883s
hyper parameter values: {'n_estimators': 2000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': False}
accuracy: 0.9523809523809523
precision: 0.9523809523809523
tpr: 0.9722222222222223
fpr: 0.00980392156862745
auc: 0.9812091503267973
PR-Curve tf.Tensor(0.6874204, shape=(), dtype=float32)
 cv index:  5
Training time: 40.01028561592102s
inference time: 0.12435436248779297s
hyper parameter values: {'n_estimators': 1400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}
accuracy: 0.85
precision: 0.85
tpr: 0.6517857142857143
fpr: 0.060897435897435896
auc:Error
PR-Curve tf.Tensor(0.6874693, shape=(), dtype=float32)
 cv index:  6


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


Training time: 39.4883873462677s
inference time: 0.1019601821899414s
hyper parameter values: {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 100, 'bootstrap': True}
accuracy: 0.55
precision: 0.55
tpr: 0.5750000000000001
fpr: 0.09247076023391813
auc: 0.7412646198830409
PR-Curve tf.Tensor(0.68726885, shape=(), dtype=float32)
 cv index:  7
Training time: 38.15528702735901s
inference time: 0.02546238899230957s
hyper parameter values: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': True}
accuracy: 0.75
precision: 0.75
tpr: 0.7633333333333333
fpr: 0.0625
auc:Error
PR-Curve tf.Tensor(0.6872347, shape=(), dtype=float32)
 cv index:  8
Training time: 39.5516939163208s
inference time: 0.09806990623474121s
hyper parameter values: {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 100, 'bootstrap': True}
accura

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  


Training time: 39.87249970436096s
inference time: 0.10397672653198242s
hyper parameter values: {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 100, 'bootstrap': True}
accuracy: 0.9682539682539683
precision: 0.9682539682539683
tpr: 0.6565656565656566
fpr: 0.02091503267973856
auc: 0.817825311942959
PR-Curve tf.Tensor(0.6875737, shape=(), dtype=float32)
 cv index:  1
Training time: 39.75717639923096s
inference time: 0.10267353057861328s
hyper parameter values: {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 100, 'bootstrap': True}
accuracy: 0.8253968253968254
precision: 0.8253968253968254
tpr: 0.6300705467372134
fpr: 0.10343915343915344
auc: 0.76331569664903
PR-Curve tf.Tensor(0.6877314, shape=(), dtype=float32)
 cv index:  2
Training time: 41.26777386665344s
inference time: 0.17737078666687012s
hyper parameter values: {'n_estimators': 2000, 'min_samples_split': 5, 'min_sa